In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 



%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_original = pd.read_csv('ElectionsData.csv', header=0)
data_original.head()

,Vote,Occupation_Satisfaction,Avg_monthly_expense_when_under_age_21,Avg_lottary_expanses,Age_group,Avg_monthly_expense_on_pets_or_plants,Looking_at_poles_results,Avg_environmental_importance,Married,Gender,...,Last_school_grades,Most_Important_Issue,Number_of_differnt_parties_voted_for,Political_interest_Total_Score,Number_of_valued_Kneset_members,Main_transportation,Occupation,Financial_agenda_matters,Num_of_kids_born_last_10_years,Overall_happiness_score
0,Violets,1.0,113.445801,1688.510190,Below_30,1477.809692,Yes,202.042787,Yes,Male,...,60.0,Financial,12.0,683.760440,1.0,Public_or_other,Industry_or_other,No,1.0,392.803101
1,Browns,2.0,161.720841,696.069582,Below_30,2969.865888,Yes,88.942300,Yes,Male,...,60.0,Financial,9.0,793.017823,2.0,Car,Industry_or_other,No,0.0,41.028616
2,Greens,6.0,524.947117,50048.853880,30-45,30803.812970,No,789.796962,Yes,Female,...,80.0,Healthcare,10.0,253.102383,1.0,Motorcycle_or_truck,Services_or_Retail,No,0.0,635.000942
3,Whites,1.0,521.454363,50265.185360,Below_30,30396.557250,No,565.727946,Yes,Male,...,NaN,Social,8.0,278.904297,1.0,Car,Services_or_Retail,No,0.0,502.278182
4,Browns,6.0,210.879361,7793.195795,30-45,5021.415879,No,423.589896,Yes,Female,...,70.0,Social,11.0,605.564038,4.0,Car,Public_Sector,No,2.0,40.469037


In [25]:
data = data_original.copy()

# Converting categorial columns to int (section 2)
ObjFeat = data.keys()[data.dtypes.map(lambda x: x == 'object')] # picks all features type object
for f in ObjFeat:
    data[f] = data[f].astype("category")
    data[f+"Int"] = data[f].cat.rename_categories(range(data[f].nunique())).astype('Int64')
    data.loc[data[f].isnull(), f+"Int"] = np.nan #fix NaN conversion
    data[f] = data[f+"Int"]
    data = data.drop(f+"Int", axis=1) # remove temporary columns


In [33]:
indices = data.isnull().values.sum(axis=1)
data[indices == 0]


,Vote,Occupation_Satisfaction,Avg_monthly_expense_when_under_age_21,Avg_lottary_expanses,Age_group,Avg_monthly_expense_on_pets_or_plants,Looking_at_poles_results,Avg_environmental_importance,Married,Gender,...,Last_school_grades,Most_Important_Issue,Number_of_differnt_parties_voted_for,Political_interest_Total_Score,Number_of_valued_Kneset_members,Main_transportation,Occupation,Financial_agenda_matters,Num_of_kids_born_last_10_years,Overall_happiness_score
2,2,6.0,524.947117,50048.853880,0,30803.812970,0,789.796962,1,0,...,80.0,4,10.0,253.102383,1.0,2,3,0,0.0,635.000942
4,1,6.0,210.879361,7793.195795,0,5021.415879,0,423.589896,1,0,...,70.0,7,11.0,605.564038,4.0,0,2,0,2.0,40.469037
5,8,7.0,446.281570,97772.779280,2,22288.453820,0,768.216797,0,1,...,50.0,6,8.0,183.171894,2.0,1,2,0,1.0,583.265611
8,9,10.0,106.805103,15808.025820,1,1313.082826,1,720.262360,1,0,...,90.0,1,13.0,92.007121,1.0,3,3,1,0.0,549.064731
13,2,4.0,429.025222,43536.037080,0,20604.412990,0,769.883557,1,0,...,60.0,2,11.0,236.297742,1.0,3,3,1,0.0,629.596049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1,8.0,353.354577,116191.444600,0,14001.057860,0,1467.471316,0,0,...,60.0,2,8.0,834.646110,5.0,0,4,1,0.0,985.829167
9993,9,8.0,541.825662,69887.415630,0,32810.058370,1,688.694562,1,0,...,80.0,4,14.0,127.518769,2.0,1,0,0,1.0,505.712925
9995,7,9.0,124.023053,26638.227290,0,1760.420762,0,1060.517225,0,0,...,80.0,4,10.0,321.858827,1.0,2,2,1,3.0,646.016048
9997,9,7.0,173.952313,20598.709440,1,3430.140470,0,690.466486,1,0,...,90.0,1,12.0,152.322399,3.0,1,1,1,0.0,494.710087


In [18]:
#Imputation

# filling using median for numeric values and most common for nominal values

# Median - numeric
numeric_feat = data.keys()[data.dtypes.map(lambda x: x == 'float64')]
data[numeric_feat] = data[numeric_feat].fillna(data[numeric_feat].dropna().median())

# Most common - nominal
nominal_feat = data.keys()[data.dtypes.map(lambda x: x == 'Int64')]
data[nominal_feat] = data[nominal_feat].fillna(data[nominal_feat].dropna().mode().iloc[0])

C:\Users\aviv_\Miniconda3\envs\cs236756\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  # Remove the CWD from sys.path while we load stuff.
